In [1]:
# Example and test bed for the constant salt ensemble
import numpy as np
from numpy import random
from simtk import openmm, unit
from simtk.openmm import app
import sys
from sys import stdout
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA

In [2]:
#Loading a premade water box:
temperature = 300*unit.kelvin
pdb = app.PDBFile('waterbox.pdb')
forcefield = app.ForceField('tip3p.xml')
system = forcefield.createSystem(pdb.topology,nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds)
integrator = openmm.LangevinIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.002*unit.picoseconds)
system.addForce(openmm.MonteCarloBarostat(1*unit.atmospheres, 300*unit.kelvin, 25))
simulation = app.Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

In [3]:
# Defining the various parameters of the constant salt simulation:
integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
nattempts_per_update = 10
iterations = 1000       # Number of rounds of MD and constant salt moves
nsteps = 1000           # Amount of MD steps per iteration (not to be confused with number of steps in NCMC)
nattempts = 1000        # Number of identity exchanges for water and ions.
delta_chem = -160*unit.kilojoule_per_mole       # The free energy penalty to turn 2 waters into Na and Cl. To be calibrated.

# Package still under constrution!!!!
import constsalt           

In [4]:
# Initialing constant salt class.
cs = constsalt.ConstantSalt(system=system,topology=pdb.topology,temperature=temperature,
                   delta_chem=delta_chem,integrator=integrator,debug=True)
        

identifyResidues: 501 HOH molecules identified.


In [5]:
cs.update(simulation.context)

old potential energy = -1607.7508363 kJ/mol
new potential energy = -1514.57261289 kJ/mol
add salt accepted
old potential energy = -1514.57261289 kJ/mol
new potential energy = -1452.8978189 kJ/mol
add salt accepted
old potential energy = -1452.8978189 kJ/mol
new potential energy = -1402.02159118 kJ/mol
add salt accepted
old potential energy = -1402.02159118 kJ/mol
new potential energy = -1124.39177093 kJ/mol
add salt rejected
old potential energy = -1402.02159118 kJ/mol
new potential energy = -1288.29301115 kJ/mol
remove salt accepted
old potential energy = -1288.29301115 kJ/mol
new potential energy = -1365.88290371 kJ/mol
remove salt accepted
old potential energy = -1365.88290371 kJ/mol
new potential energy = -1461.97467377 kJ/mol
add salt accepted
old potential energy = -1461.97467377 kJ/mol
new potential energy = -1453.39918964 kJ/mol
remove salt accepted
old potential energy = -1453.39918964 kJ/mol
new potential energy = -1607.74888324 kJ/mol
remove salt accepted
old potential energ

In [6]:
cs.getAcceptanceProbability()

0.56

In [7]:
cs.identifyResidues(pdb.topology,("HOH","Na+","Cl-"))

identifyResidues: 501 HOH molecules identified.


[<Residue 0 (HOH) of chain 0>,
 <Residue 1 (HOH) of chain 0>,
 <Residue 2 (HOH) of chain 0>,
 <Residue 3 (HOH) of chain 0>,
 <Residue 4 (HOH) of chain 0>,
 <Residue 5 (HOH) of chain 0>,
 <Residue 6 (HOH) of chain 0>,
 <Residue 7 (Cl-) of chain 0>,
 <Residue 8 (HOH) of chain 0>,
 <Residue 9 (HOH) of chain 0>,
 <Residue 10 (HOH) of chain 0>,
 <Residue 11 (HOH) of chain 0>,
 <Residue 12 (HOH) of chain 0>,
 <Residue 13 (HOH) of chain 0>,
 <Residue 14 (HOH) of chain 0>,
 <Residue 15 (HOH) of chain 0>,
 <Residue 16 (HOH) of chain 0>,
 <Residue 17 (HOH) of chain 0>,
 <Residue 18 (HOH) of chain 0>,
 <Residue 19 (HOH) of chain 0>,
 <Residue 20 (HOH) of chain 0>,
 <Residue 21 (HOH) of chain 0>,
 <Residue 22 (HOH) of chain 0>,
 <Residue 23 (HOH) of chain 0>,
 <Residue 24 (HOH) of chain 0>,
 <Residue 25 (HOH) of chain 0>,
 <Residue 26 (HOH) of chain 0>,
 <Residue 27 (HOH) of chain 0>,
 <Residue 28 (HOH) of chain 0>,
 <Residue 29 (HOH) of chain 0>,
 <Residue 30 (HOH) of chain 0>,
 <Residue 31 (HOH)